In [1]:
import mbuild as mb
import numpy as np

In [2]:
ethane = mb.load("CC", smiles=True)
#ethane.visualize().show()

In [3]:
def adjust_bond(i, j, compound, length, relative=False):
    if relative:
        assert length >= 0, "Relative lengths must be positive"
    clone = mb.clone(compound)
    orientation = clone[j].pos - clone[i].pos
    orig_length = np.linalg.norm(orientation)
    orientation_unit = orientation / orig_length 
    clone.remove_bond((clone[i], clone[j]))
    left = [l for l in clone.bond_graph.connected_components() if clone[i] in l]
    right = [l for l in clone.bond_graph.connected_components() if clone[j] in l]
    if left:
        left = left[0]
    else:
        left = [clone[i]]
    if right:
        right = right[0]
    else:
        right = [clone[j]]
        
    if relative:
        adjust = length*orig_length - orig_length
    else:
        adjust = length - orig_length
    for p in right:
        p.pos += orientation_unit * adjust
    clone.add_bond((clone[i], clone[j]))
    return clone

In [4]:
ethane.visualize().show()
eth = adjust_bond(0, 1, ethane, length=3)
eth.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
clone_eth = mb.clone(ethane)
for p in clone_eth[0,2]:
    p.name = "x"
for p in clone_eth[1,5]:
    p.name = "br"
clone_eth.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
def rotation_matrix(angle, direction, point=None):
    """Return matrix to rotate about axis defined by point and direction.
    taken from
    https://github.com/matthew-brett/transforms3d/blob/master/original/transformations.py
    """
    sina = np.sin(angle)
    cosa = np.cos(angle)
    # rotation matrix around unit vector
    R = np.diag([cosa, cosa, cosa])
    R += np.outer(direction, direction) * (1.0 - cosa)
    direction *= sina
    R += np.array([[          0.0, -direction[2],  direction[1]],
                   [ direction[2],           0.0, -direction[0]],
                   [-direction[1],  direction[0],           0.0]])
    M = np.identity(4)
    M[:3, :3] = R
    if point is not None:
        # rotation not around origin
        M[:3, 3] = point - np.dot(R, point)
    return M

In [7]:
#                   0, 1, 2, 5  (0,2) (1,5)
def adjust_dihedral(i, j, k, l, compound, angle):
    clone = mb.clone(compound)
    ik = clone[k].pos - clone[i].pos
    ik /= np.linalg.norm(ik)
    jl = clone[l].pos - clone[j].pos
    jl /= np.linalg.norm(jl)
    
    # vector used for plane normal
    ij = clone[j].pos - clone[i].pos
    ij /= np.linalg.norm(ij)
    
    proj_ik = ik - np.dot(ik, ij) * ij
    proj_ik /= np.linalg.norm(proj_ik)
    proj_jl = jl - np.dot(jl, ij) * ij
    proj_jl /= np.linalg.norm(proj_jl)

    current_angle = np.arccos(np.dot(proj_ik, proj_jl))
    diff = angle - current_angle

    clone.remove_bond((clone[i], clone[j]))
    left = [l for l in clone.bond_graph.connected_components() if clone[i] in l]
    right = [l for l in clone.bond_graph.connected_components() if clone[j] in l]
    if left:
        left = left[0]
    else:
        left = [clone[i]]
    if right:
        right = right[0]
    else:
        right = [clone[j]]
        
    rot_mat = rotation_matrix(diff, ij, point=clone[j].pos)
    for p in right:
        p.pos = np.dot(rot_mat[:3,:3], p.pos) + rot_mat[:,3][:3]
    clone.add_bond((clone[i], clone[j]))
    return clone

In [8]:
ethane.visualize().show()
eth = adjust_dihedral(0, 1, 2, 5, ethane, 0) #np.pi/3)
eth.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [9]:
clone_eth = mb.clone(ethane)
for p in clone_eth[0,1,5]:
    p.name = "x"
clone_eth.visualize().show()

#   0 - 1
#        \
#         5

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [10]:
#                1, 0, 5
def adjust_angle(i, j, k, compound, angle):
    """
      i - j
     /
    k
    """
    clone = mb.clone(compound)
    ik = clone[k].pos - clone[i].pos
    ik /= np.linalg.norm(ik)
    
    ij = clone[j].pos - clone[i].pos
    ij /= np.linalg.norm(ij)
    
    norm = np.cross(ij, ik) 
    norm /= np.linalg.norm(norm)

    current_angle = np.arccos(np.dot(ij, ik))
    diff = angle - current_angle

    clone.remove_bond((clone[i], clone[j]))
    clone.remove_bond((clone[i], clone[k]))
    left = [l for l in clone.bond_graph.connected_components() if clone[j] in l]
    right = [l for l in clone.bond_graph.connected_components() if clone[k] in l]
    if left:
        left = left[0]
    else:
        left = [clone[j]]
    if right:
        right = right[0]
    else:
        right = [clone[k]]
    
    rot_mat = rotation_matrix(diff, norm, point=clone[i].pos)
    for p in right:
        p.pos = np.dot(rot_mat[:3,:3], p.pos) + rot_mat[:,3][:3]
    clone.add_bond((clone[i], clone[j]))
    clone.add_bond((clone[i], clone[k]))
    return clone

In [12]:
#ethane.visualize().show()
eth = adjust_angle(1, 0, 5, ethane, np.pi)
eth.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol